In [4]:
import SimpleITK as sitk
import nibabel as nib
import os
import numpy as np
import matplotlib.pyplot as plt

root="/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA"

In [46]:
def read_data(path):
    nii=nib.load(path)
    data=np.asanyarray(nii.dataobj)
    
    return nii, data

def save_nii(data, affine, path2save, header=None):
    nii2save=nib.Nifti1Image(data, affine, header=header)
    nib.save(nii2save, path2save)

def reshaper(data_path, new_shape, reshaped_path, affine=np.array([]), header=0):
    nii_data, data = read_data(data_path)
    data_reshaped=np.reshape(data, new_shape)

    # data_reshaped.set_data_dtype(np.int16)
    
    if not affine.any():
        affine = nii_data.affine

    if not header:
        header = nii_data.header
        
    save_nii(data_reshaped, affine, reshaped_path, header=header)

In [29]:
filename="T2starMapMGE-anterior-reshaped.nii.gz"
reshaped_path = os.path.join(root,animal_id,"mri",session,"anat",filename)
nii=nib.load(reshaped_path)


nib.save(nii.set_data_dtype(np.int16), reshaped_path)

AttributeError: 'NoneType' object has no attribute 'to_filename'

In [47]:
animal_id="rEO_08"
session="ses-postsurgery20240808"
# bids_work="_ind_type_40"

filename="T2starMapMGE-posterior.nii.gz"
data_path=os.path.join(root,animal_id,"mri",session,"anat",filename)
filename="T2starMapMGE-posterior-reshaped.nii.gz"
reshaped_path = os.path.join(root,animal_id,"mri",session,"anat",filename)
nii_fixed, _ = read_data(data_path)

filename="T2starMapMGE-anterior.nii.gz"
data_path=os.path.join(root,animal_id,"mri",session,"anat",filename)
filename="T2starMapMGE-anterior-reshaped.nii.gz"
reshaped_path = os.path.join(root,animal_id,"mri",session,"anat",filename)

reshaper(data_path, (256,256,1,16), reshaped_path, nii_fixed.affine, nii_fixed.header)

# filename = "_TurboRARE_20240808143311_30001.nii.gz"
# fixed_path = os.path.join(root,animal_id,"mri",session,"anat",filename)


# reshaper(data_path, (256,256,1,16), reshaped_path, nii_fixed.affine, nii_fixed.header)

# rEO_10

### Reslicing, Fixing the T2* MGE dimensions

In [19]:
animal_id="rEO_10"
session="ses-postsurgery20241024"
# bids_work="_ind_type_40"

moving_filename="sub-rEO_10_ses-postsurgery20241024_task-rightHemi_acq-T2starMapMGE_T2w-ind_8.nii.gz"
moving_path=os.path.join(root,animal_id,"mri",session,"anat",moving_filename)
nii_moving, data_moving = read_data(moving_path)

# Penta MRID T2star MGE, 
data_reshaped=np.reshape(data_moving, (256,256,1,8))
# data_single=np.reshape(data_moving[:,:,4], (256,256,1,1))
nii_img=nii_moving.slicer[:,:,3:4]

newfilename=".".join([moving_filename.split(".")[0]+"-reshaped", moving_filename.split(".")[1], moving_filename.split(".")[2]])
newpath = os.path.join(root,animal_id,"mri",session,"anat", newfilename)

save_nii(data_reshaped, nii_img.affine, newpath, header=nii_img.header)

In [26]:
data_new = data_single
new_filename="duo-sagittal-anat-reshaped.nii.gz"
new_path=os.path.join(root,animal_id,"mri","ses-"+session,"anat",new_filename)

save_nii(data_new, nii_img.affine, new_path, header=nii_img.header)

In [12]:
segmentation_filename = "penta_segmentation_coronal.nii.gz"
segm_path = os.path.join(root,animal_id,"mri","ses-"+session,"anat",segmentation_filename)

nii_segm, data_segm = read_data(segm_path)

In [13]:
segm_singleSlice = np.reshape(data_segm[:,:,-1], (256,256,1))

segmentationNew_filename = "penta-segmentation_coronal-reshaped.nii.gz"
segmNew_path = os.path.join(root,animal_id,"mri","ses-"+session,"anat", segmentationNew_filename)

save_nii(segm_singleSlice, nii_img.affine, segmNew_path, header=nii_img.header)

### Saving the Relaxation Heatmap as nii.gz with correct affine

In [12]:
relaxation_heatmap="relaxation_heatmap-sagittal-right_bundle.npy"
relaxation_path=os.path.join(root,animal_id,"mri","ses-"+session,relaxation_heatmap)
heatmap = np.load(relaxation_path)
heatmap=np.reshape(heatmap, (np.shape(heatmap)[0], np.shape(heatmap)[1], 1))

heatmap_nii="relaxation_heatmap-sagittal-right_bundle.nii.gz"
relaxation_path=os.path.join(root,animal_id,"mri","ses-"+session,"anat",heatmap_nii)
save_nii(heatmap, nii_img.affine, relaxation_path, header=nii_img.header)

# Warping with SimpleITK .txt Transformations

In [26]:
identity_filename = "identity_transform.txt"
session="postsurgery20241024"

identity_path = os.path.join(root,animal_id,"mri","ses-"+session,"anat",identity_filename)
identityTx = sitk.ReadTransform(identity_path)


In [39]:
# read the fixed img, moving img, affine txt, output from the command line arguments

session="postsurgery20241024"
fixed_img_filename = "sub-rEO_10_ses-postsurgery20241024_task-sagittal_acq-T2TurboRARE_T2w-ind_1_resampledAligned.nii.gz"
fixed_path = os.path.join(root,animal_id,"mri","ses-"+session,"anat",fixed_img_filename)

# session="postsurgery20240924"
moving_filename="duo-relaxation_heatmap-sagittal-right_bundle.nii.gz"
moving_path=os.path.join(root,animal_id,"mri","ses-"+session,"anat",moving_filename)

resampled_filename = "duo-relaxation_heatmap-sagittal-right_bundle-aligned_to_alignedSagittal.nii.gz"
resampled_path = os.path.join(root,animal_id,"mri","ses-"+session,"anat",resampled_filename)

transform_filename = "transformation-sagittalAligned_to_rightHemiT2sMGEsingleSlice.txt"
transform_path = os.path.join(root,animal_id,"mri","ses-"+session,"anat",transform_filename)


In [40]:
# Use Nearest Neighbor Interpolater if resampling Segmentations; to not generate new labels.
fixed_img = sitk.ReadImage(fixed_path)
moving_img = sitk.ReadImage(moving_path)
tx = sitk.ReadTransform(transform_path)
inverseTx = tx.GetInverse()
nn_interpolator = sitk.sitkNearestNeighbor

resampled_img = sitk.Resample(moving_img, fixed_img, inverseTx)
sitk.WriteImage(resampled_img, resampled_path)